In [5]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn import linear_model

ImportError: No module named 'xgboost'

Read input Audio, Video, Transcript features. 

In [6]:
opensmile_std_input_df = pd.read_excel("./Audio/open_face_accoustic_std.xlsx")
opensmile_mean_input_df = pd.read_excel("./Audio/open_face_accoustic_means.xlsx")
transcript_input_df = pd.read_csv("./Transcript/segment_annotation_transcript_features.csv")
video_max_df = pd.read_csv("./Video/feature_annotation_max.csv")
video_min_df = pd.read_csv("./Video/feature_annotation_min.csv")
video_mean_df = pd.read_csv("./Video/feature_annotation_mean.csv")
video_std_df = pd.read_csv("./Video/feature_annotation_sd.csv")

In [7]:
for input_df in [opensmile_std_input_df,opensmile_mean_input_df,transcript_input_df,video_max_df,video_min_df,video_mean_df,video_std_df]:
    print(input_df.shape)

(1352, 78)
(1352, 78)
(1352, 16)
(1352, 63)
(1352, 63)
(1352, 63)
(1352, 63)


In [4]:
transcript_input_df.head()

,video,laughter_start,laughter_end,laughter_value,start_segment_s,end_segment_s,start_segment_frame,end_segment_frame,number_of_words,number_of_elongated_words,number_of_continuous_segments,number_of_pauses,max_pause_time,min_pause_time,avg_pause_time,last_pause_length
0,AndyWoodhull,5.0,8.0,1,0.0,5.0,1,150,13,6,2,1,1.70,0.84,1.2700,1.70
1,AndyWoodhull,10.0,12.0,1,8.0,10.0,241,300,4,1,0,0,0.00,0.00,0.0000,0.00
2,AndyWoodhull,20.0,21.0,1,12.0,20.0,361,600,29,7,2,1,1.84,0.69,1.2650,1.84
3,AndyWoodhull,47.0,48.0,1,21.0,47.0,631,1410,97,21,4,3,1.64,0.42,0.8875,1.64
4,AndyWoodhull,57.0,62.0,1,48.0,57.0,1441,1710,33,9,2,1,1.00,0.70,0.8500,1.00


In [9]:
transcript_video = list(transcript_input_df.video)
transcript_laughter_start = list(transcript_input_df.laughter_start)
transcript_video_laughter = zip(transcript_video,transcript_laughter_start)
a = set(list(transcript_video_laughter))

opensmile_video = list(opensmile_std_input_df.video)
opensmile_laughter_start = list(opensmile_std_input_df.laughter_start)
opensmile_video_laughter = zip(opensmile_video,opensmile_laughter_start)
b = set(list(opensmile_video_laughter))

#a - b

In [10]:
transcript_input_df.columns

Index(['video', 'laughter_start', 'laughter_end', 'laughter_value',
       'start_segment_s', 'end_segment_s', 'start_segment_frame',
       'end_segment_frame', 'number_of_words', ' number_of_elongated_words',
       ' number_of_continuous_segments', ' number_of_pauses',
       ' max_pause_time ', ' min_pause_time', ' avg_pause_time',
       ' last_pause_length'],
      dtype='object')

In [11]:
transcript_features = ['number_of_words', ' number_of_elongated_words',' number_of_continuous_segments',
                       ' number_of_pauses', ' max_pause_time ', ' min_pause_time', ' avg_pause_time',
               ' last_pause_length']

In [13]:
transcript_X = transcript_input_df[transcript_features]

In [14]:
transcript_X.columns = ['number_of_words', 'number_of_elongated_words','number_of_continuous_segments',
                       'number_of_pauses', ' max_pause_time ', 'min_pause_time', 'avg_pause_time',
               'last_pause_length']

In [15]:
transcript_X

,number_of_words,number_of_elongated_words,number_of_continuous_segments,number_of_pauses,max_pause_time,min_pause_time,avg_pause_time,last_pause_length
0,13,6,2,1,1.70,0.84,1.2700,1.70
1,4,1,0,0,0.00,0.00,0.0000,0.00
2,29,7,2,1,1.84,0.69,1.2650,1.84
3,97,21,4,3,1.64,0.42,0.8875,1.64
4,33,9,2,1,1.00,0.70,0.8500,1.00


In [17]:
transcript_y = transcript_input_df.laughter_value

In [37]:
kf = KFold(n_splits=5)
kf.get_n_splits(transcript_X)

data = np.array(transcript_X)
output = np.array(transcript_y)

experiment = 0 



for train_index, test_index in kf.split(transcript_X):
    
    print('experiment : ', experiment)
    
    experiment = experiment + 1 
    
    train_data = data[train_index]
    train_output = output[train_index]
    
    kf1 = KFold(n_splits=4)
    kf1.get_n_splits(train_data)
    
    fold = 1 
    
    max_accuracy = 0 
    c_max_accuracy = 0 
    
    C_Accuracies  = {}
    
    for tr_index, validation_index in kf1.split(train_data):
        
        print('Fold : ', fold)
        fold = fold + 1
        
        tr_data = train_data[tr_index]
        tr_output = output[tr_index]
        
        validation_data = train_data[validation_index]
        validation_output = output[validation_index]
        
        
        C_list = [1e0,1e1,1e2,1e3,1e4,1e5]
        
        for c in C_list:
            logreg = linear_model.LogisticRegression(C=1e5)
        
            logreg.fit(tr_data, tr_output )

            pred = logreg.predict(validation_data)
            
            accuracy = accuracy_score(pred,validation_output)
            #print('c : ',c)
            #print('Accuracy_Score : ', accuracy )
            
            if c in C_Accuracies:
                C_Accuracies[c].append(accuracy)
            else:
                C_Accuracies[c] = [accuracy]
            

    for c in C_Accuracies:
        accuracy = np.mean(C_Accuracies[c])
        
        if max_accuracy < accuracy:
            c_max_accuracy = c
            max_accuracy = accuracy
            
    test_data = data[test_index]
    test_output = output[test_index]
    
    logreg = linear_model.LogisticRegression(C=c_max_accuracy)
    
    logreg.fit(train_data, train_output)
    
    pred = logreg.predict(test_data)
    
    print("max_c : ", c_max_accuracy)
    print("max_validation_accuracy : " , max_accuracy)
    
    print('Test Accuracy_Score', accuracy_score(pred,test_output))
    
    print('*'*10)


experiment :  0
Fold :  1
Fold :  2
Fold :  3
Fold :  4
max_c :  10000.0
max_validation_accuracy :  0.210997032165
Test Accuracy_Score 0.229927007299
**********
experiment :  1
Fold :  1
Fold :  2
Fold :  3
Fold :  4
max_c :  10000.0
max_validation_accuracy :  0.229285313227
Test Accuracy_Score 0.193430656934
**********
experiment :  2
Fold :  1
Fold :  2
Fold :  3
Fold :  4
max_c :  10000.0
max_validation_accuracy :  0.252138980241
Test Accuracy_Score 0.29197080292
**********
experiment :  3
Fold :  1
Fold :  2
Fold :  3
Fold :  4
max_c :  10000.0
max_validation_accuracy :  0.242069062325
Test Accuracy_Score 0.361313868613
**********
experiment :  4
Fold :  1
Fold :  2
Fold :  3
Fold :  4
max_c :  10000.0
max_validation_accuracy :  0.263686131387
Test Accuracy_Score 0.40293040293
**********


In [47]:
data = np.array(transcript_input_df[transcript_features])
output = np.array(transcript_input_df.laughter_value)


In [53]:
dtrain = xgb.DMatrix( data[330:],output[330:])
dtest = xgb.DMatrix(data[:330],output[:330])

In [70]:
params  = {'objective': 'reg:logistic',
          'num_class': 3
          } 
param['nthread'] = 4
param['eval_metric'] = 'auc'

In [71]:
#param['eval_metric'] = ['auc', 'ams@0']

# alternatively:
plst = param.items()
#plst = plst + [('eval_metric', 'ams@0')]

In [72]:
evallist  = [(dtest,'eval'), (dtrain,'train')]

In [73]:
num_round = 10
bst = xgb.train( plst, dtrain, num_round, evallist )

XGBoostError: b'[19:07:48] src/objective/regression_obj.cc:108: label must be in [0,1] for logistic regression'

{0, 1, 2}